In [1]:
import numpy as np
import networkx as nx

In [34]:
seq1=''
with open("seq1.txt", "r") as conf_file:
    for idx, line in enumerate(conf_file):
        if idx==0:
            continue
        seq1 = seq1.join(line.replace("\n", ""))
        if len(seq1)>= 15:
            seq1 = seq1[:15]
            break
        print(len(line))
#         name, val = line.split("=")
#         config[name.strip()] = int(val.replace("\n", "").strip())

In [36]:
len(seq1)

15

In [2]:
# config:
same = 5
diff = -5
gp = -2
max_number_paths = 10 # ile wypiszemy maksymalnie różnych dopasowań
max_seq_length = 15 #maksyamlna długość sekwencji, to ma być tylko sprawdzenie


# odpalenie: pyhton plik.py -a input1 -b intput2 -c config

In [10]:
seq1 = list('SMART')
seq2 = list('MARS')

In [41]:
def create_matrix_and_graph(seq1,seq2):
    n = len(seq1)
    m = len(seq2)
    matrix = [[0]*(n+1) for i in range(m+1)]
#     stworz graf
    DG = nx.DiGraph()
    for i in range(n+1):
        for j in range(m+1):
            DG.add_node(str(j)+','+str(i))
            
#     wypełnij wartości macierzy i dodaj odpowiednie krawędzie w grafie
    for j in range(1,m+1):
        matrix[j][0]=matrix[j-1][0]+gp
        DG.add_edge(str(j)+',0',str(j-1)+',0')
        for i in range(1,n+1):
            matrix[0][i]=matrix[0][i-1]+gp
            DG.add_edge('0,'+str(i),'0,'+str(i-1))
            
            val1 = matrix[j-1][i]+gp
            val2 = matrix[j][i-1]+gp
            if seq1[i-1] == seq2[j-1]:
                val3 = matrix[j-1][i-1]+same
            else:
                val3 = matrix[j-1][i-1]+diff
            val_max = max([val1,val2,val3])
            matrix[j][i] = val_max
            
            if val1 == val_max:
                DG.add_edge(str(j)+','+str(i), str(j-1)+','+str(i))
            if val2 == val_max:
                DG.add_edge(str(j)+','+str(i), str(j)+','+str(i-1))
            if val3 == val_max: 
                DG.add_edge(str(j)+','+str(i), str(j-1)+','+str(i-1))
    return matrix, DG

In [42]:
macierz, graf = create_matrix_and_graph(seq1,seq2)

In [47]:
sciezki = paths_in_graph(graf, len(seq2), len(seq1), max_number_paths)

In [48]:
sciezki_ = change_str_to_int(sciezki)

In [52]:
modification_to_file(sciezki_,seq1,seq2,'test.txt', macierz)

In [45]:
def paths_in_graph(DG,m,n,max_number_paths):
    x = []
    for num, i in enumerate(nx.all_simple_paths(DG,str(m)+','+str(n),'0,0')):
        if num == max_number_paths:
            break
        x.append([])
        for j in i:
            x[num].append(j.split(','))
    return x

In [46]:
def change_str_to_int(x):
    for idx_x, lists in enumerate(x):
        for idx_lists, node in enumerate(lists):
            for idx_node, number in enumerate(node):
                x[idx_x][idx_lists][idx_node] = int(number)
        x[idx_x] = x[idx_x][::-1]
    return x

In [51]:
def modification_to_file(paths, seq1, seq2, filename, matrix):
    file = open(filename,'w')
    sums=0
    for idx in paths[0]:
        sums+=matrix[idx[0]][idx[1]]   
    file.write('SCORE = '+str(sums)+'\n')
    
    for path in paths:
        line1, line2 = modification(path, seq1, seq2)
        file.write(line1+'\n')
        file.write(line2+'\n')
        file.write("\n")
    file.close()
        
def modification(path, seq1, seq2):
    line1 = ""
    line2 = ""
    for idx_node, node in enumerate(path[1:],1):
        if sum(path[idx_node-1])+2==sum(path[idx_node]):
            #wtedy obie są dobre
            line1+=seq1[path[idx_node-1][1]]
            line2+=seq2[path[idx_node-1][0]]
        elif path[idx_node-1][0]==path[idx_node][0]:
            #wtedy w drugim seq jest gap
            line1+=seq1[path[idx_node-1][1]]
            line2+='-'
        else:
            line1+='-'
            line2+=seq2[path[idx_node-1][0]]
    return line1, line2


In [53]:
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("square", type=int,
                    help="display a square of a given number")
parser.add_argument("-v", "--verbosity", type=int,
                    help="increase output verbosity")
args = parser.parse_args()
answer = args.square**2
if args.verbosity == 2:
    print "the square of {} equals {}".format(args.square, answer)
elif args.verbosity == 1:
    print "{}^2 == {}".format(args.square, answer)
else:
    print answer